In [1]:
import os, sys, json, random
import pandas as pd
import numpy as np

from scipy.stats import pearsonr
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import chi2
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV

import matplotlib.pyplot as plt

In [2]:
BASEDIRECTORY = "/Users/phillip/Sync/MIMIC III Uncompressed Datasets"

In [3]:
#  Possible feature names in D_ITMES.csv
features_list = [
    "Heart Rate",
    "O2 saturation pulseoxymetry",
    "Temperature Celsius",
    "Temperature C",
    "Temperature C (calc)",
    "Arterial Blood Pressure systolic",
    "ABP [Systolic]",
    "Arterial BP [Systolic]",
    "Arterial Blood Pressure diastolic",
    "ABP [Diastolic]",
    "Arterial BP [Diastolic]",
    "Arterial Blood Pressure mean",
    "Arterial BP Mean",
    "Respiratory Rate (Total)",
    "Respiratory Rate",
    "EtCO2",
    "Base Excess",
    "HCO3",
    "FIO2",
    "FiO2" "Inspired O2 Fraction",
    "ph level",
    "PH",
    "Ph",
    "Arterial PaCO2",
    "Arterial O2 Saturation",
    "SaO2",
    "Alkaline Phosphatase",
    "Alkaline Phosphate",
    "Calcium",
    "Chloride (whole blood)",
    "Chloride",
    "Creatinine",
    "Direct Bilirubin",
    "Glucose (serum)",
    "Lactic Acid",
    "Magnesium",
    "Potassium",
    "Troponin",
    "Hematocrit",
    "Hemoglobin",
    "PTT",
    "Ptt",
    "WBC",
    "FIBRINOGEN",
    "Fibrinogen",
    "Platelets",
    "Present Weight  (kg)",
    "Weight Kg",
]

In [4]:
d_items_df = pd.read_csv("{}/{}".format(BASEDIRECTORY, "D_ITEMS.csv"))
diagnose_df = pd.read_csv("{}/{}".format(BASEDIRECTORY, "DIAGNOSES_ICD.csv"))
d_icd_diagnose_df = pd.read_csv("{}/{}".format(BASEDIRECTORY, "D_ICD_DIAGNOSES.csv"))
admission_df = pd.read_csv("{}/{}".format(BASEDIRECTORY, "ADMISSIONS.csv"))
patient_df = pd.read_csv("{}/{}".format(BASEDIRECTORY, "PATIENTS.csv"))

In [58]:
patient_df.head(10).to_csv(
    "/Users/phillip/Documents/PhD/PhD3/Docs/Thesis/sample data for appendix/sample_PATIENTS.csv"
)

In [5]:
selected_itemids_df = d_items_df[
    d_items_df.LABEL.isin(features_list) & (d_items_df.LINKSTO == "chartevents")
][["ITEMID", "LABEL"]]

In [6]:
# 找出sepsis相关的ICD码
sepsis_icd = d_icd_diagnose_df[
    d_icd_diagnose_df.SHORT_TITLE.str.contains("sepsis|septic")
]
sepsis_icd

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
69,242,0031,Salmonella septicemia,Salmonella septicemia
542,593,0545,Herpetic septicemia,Herpetic septicemia
595,646,0380,Streptococcal septicemia,Streptococcal septicemia
596,647,03810,Staphylcocc septicem NOS,"Staphylococcal septicemia, unspecified"
598,649,03812,MRSA septicemia,Methicillin resistant Staphylococcus aureus se...
599,650,03819,Staphylcocc septicem NEC,Other staphylococcal septicemia
600,651,0382,Pneumococcal septicemia,Pneumococcal septicemia [Streptococcus pneumon...
601,652,0383,Anaerobic septicemia,Septicemia due to anaerobes
602,653,03840,Gram-neg septicemia NOS,"Septicemia due to gram-negative organism, unsp..."
603,654,03841,H. influenae septicemia,Septicemia due to hemophilus influenzae [H. in...


In [7]:
selected_ICD9 = ["99592", "77181"]

In [8]:
selected_chartevents_df = pd.read_csv("selected_chartevents.csv")

In [10]:
selected_chartevents_df.head(50)

,Unnamed: 0,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,VALUENUM
0,0,36,165660,241249.0,225668,2134-05-12 12:35:00,2134-05-12 12:38:00,1.0
1,1,36,165660,241249.0,220045,2134-05-12 13:00:00,2134-05-12 13:55:00,86.0
2,2,36,165660,241249.0,220210,2134-05-12 13:00:00,2134-05-12 13:55:00,21.0
3,3,36,165660,241249.0,220277,2134-05-12 13:00:00,2134-05-12 13:55:00,93.0
4,4,36,165660,241249.0,220045,2134-05-12 14:00:00,2134-05-12 14:39:00,85.0
5,5,36,165660,241249.0,220210,2134-05-12 14:00:00,2134-05-12 14:39:00,19.0
6,6,36,165660,241249.0,220277,2134-05-12 14:00:00,2134-05-12 14:39:00,91.0
7,7,36,165660,241249.0,220045,2134-05-12 15:00:00,2134-05-12 16:50:00,87.0
8,8,36,165660,241249.0,220210,2134-05-12 15:00:00,2134-05-12 16:51:00,20.0
9,9,36,165660,241249.0,220277,2134-05-12 15:00:00,2134-05-12 16:51:00,91.0


In [11]:
selected_admissionID_series = selected_chartevents_df.HADM_ID.drop_duplicates()
selected_subjectID_series = selected_chartevents_df.SUBJECT_ID.drop_duplicates()
# print(len(selected_admissionID_series), len(selected_subjectID_series))

In [12]:
grouped = (
    selected_chartevents_df.groupby("HADM_ID")
    .apply(lambda x: x.sort_values("CHARTTIME"))
    .reset_index(drop=True)
)

In [13]:
g = grouped.groupby("HADM_ID")

In [14]:
def combineColumns(
    df: "target dataframe",
    columns_list: "list of column names",
    newColumn: "name of merged column",
) -> "pd.series":
    real_list = list(filter(lambda x: x in df.columns, columns_list))
    #     print(real_list)
    if real_list:
        n_notNA_series = df[real_list].shape[1] - df[real_list].isna().sum(axis=1)
        ret = (
            df[real_list]
            .fillna(0)
            .sum(axis=1)
            .replace(0, np.nan)
            .divide(n_notNA_series)
        )
        df.drop(columns=real_list, inplace=True)
        df[newColumn] = ret

In [15]:
# 合并相同的含义的列
def merge(df):
    combineColumns(
        df,
        ["Temperature Celsius", "Temperature C", "Temperature C (calc)"],
        "Temperature",
    )
    combineColumns(
        df,
        [
            "ABP [Systolic]",
            "Arterial BP [Systolic]",
            "Arterial Blood Pressure systolic",
        ],
        "ABP-S",
    )
    combineColumns(
        df,
        [
            "ABP [Diastolic]",
            "Arterial BP [Diastolic]",
            "Arterial Blood Pressure diastolic",
        ],
        "ABP-D",
    )
    combineColumns(df, ["Arterial BP Mean", "Arterial Blood Pressure mean"], "ABP-M")
    combineColumns(df, ["Respiratory Rate", "Respiratory Rate (Total)"], "RR")
    combineColumns(
        df, ["ph level", "PH", "Ph",], "PH",
    )
    combineColumns(df, ["Present Weight  (kg)", "Weight Kg",], "weight")
    combineColumns(
        df, ["Arterial O2 Saturation", "O2 saturation pulseoxymetry", "SaO2"], "SaO2"
    )
    duplicated_columns = df.columns[df.columns.duplicated()]
    for c in duplicated_columns:
        combineColumns(df, [c], c)

In [16]:
x = g.get_group(187556)[["ITEMID", "CHARTTIME", "VALUENUM"]]
print(len(x), len(x.drop_duplicates()))
print(x.drop_duplicates())
# print(x.drop_duplicates().pivot(index="CHARTTIME", columns="ITEMID", values="VALUENUM"))

307 304
          ITEMID            CHARTTIME    VALUENUM
30042149     211  2192-11-12 19:45:00   71.000000
30042150     618  2192-11-12 19:45:00   21.000000
30042151     677  2192-11-12 20:00:00   35.888901
30042152     618  2192-11-12 20:00:00   15.000000
30042153     211  2192-11-12 20:00:00   64.000000
...          ...                  ...         ...
30042451     618  2192-11-14 12:00:00   26.000000
30042452     211  2192-11-14 12:00:00  100.000000
30042453     211  2192-11-14 13:00:00   90.000000
30042454     677  2192-11-14 13:00:00   36.333302
30042455     618  2192-11-14 13:00:00   26.000000

[304 rows x 3 columns]


In [19]:
df_list = []
c = 0
for admission_id in selected_admissionID_series:
    #         print(admission_id)
    try:
        temp = (
            g.get_group(admission_id)[["CHARTTIME", "ITEMID", "VALUENUM"]]
            .drop_duplicates()
            .pivot(index="CHARTTIME", columns="ITEMID", values="VALUENUM")
        )
    except ValueError:
        continue
    temp.columns = list(
        map(
            lambda x: str(x).replace(
                str(x),
                selected_itemids_df.LABEL[selected_itemids_df.ITEMID == x].iloc[0],
            ),
            temp.columns,
        )
    )
    #     print(patientid)
    temp.reset_index(inplace=True)
    merge(temp)
    # 加入静态的变量
    # get SUBJECT_ID first
    sub_id = admission_df[admission_df.HADM_ID == admission_id].SUBJECT_ID.values[0]
    # gender
    temp["GENDER"] = patient_df[patient_df.SUBJECT_ID == sub_id].GENDER.values[0]
    # age
    dob = pd.to_datetime(patient_df[patient_df.SUBJECT_ID == sub_id].DOB.values[0])
    # 小于1990年代表年龄大于等于90，一律按90岁算
    if dob.year < 1990:
        temp["AGE"] = 32850
    else:
        temp["AGE"] = (pd.to_datetime(temp.CHARTTIME) - dob) / pd.Timedelta(1, "D")
    # ICULOS
    doa = pd.to_datetime(
        admission_df[admission_df.HADM_ID == admission_id].ADMITTIME.values[0]
    )
    temp["ICULOS"] = (pd.to_datetime(temp.CHARTTIME) - doa) / pd.Timedelta(1, "hour")
    # 加入结果标签
    if (
        diagnose_df[diagnose_df.HADM_ID == admission_id].ICD9_CODE == selected_ICD9[0]
    ).sum() > 0:
        temp["LABEL"] = 1
    elif (
        diagnose_df[diagnose_df.HADM_ID == admission_id].ICD9_CODE == selected_ICD9[1]
    ).sum() > 0:
        temp["LABEL"] = 2
    else:
        temp["LABEL"] = 0
    # 把处理好的每一个dataframe加入到df_list列表中
    df_list.append(temp)
    if c % 10000 == 0:
        print(c)
    c += 1
#     sys.exit(1)

0
10000
20000
30000
40000
50000


In [21]:
len(df_list)

56884

In [52]:
df_list[3]

,GENDER,AGE,ICULOS,Heart Rate,RR,SaO2
0,1,28068.883333,2.383333,136.0,39.0,NaN
1,1,28068.884028,2.400000,NaN,NaN,99.0
2,1,28068.895833,2.683333,132.0,35.0,100.0
3,1,28068.916667,3.183333,131.0,32.0,98.0
4,1,28068.958333,4.183333,125.0,36.0,98.0
5,1,28069.000000,5.183333,85.0,33.0,99.0
6,1,28069.041667,6.183333,105.0,31.0,96.0
7,1,28069.083333,7.183333,106.0,29.0,97.0
8,1,28069.125000,8.183333,85.0,22.0,99.0
9,1,28069.129167,8.283333,NaN,NaN,NaN


In [31]:
for i in range(len(df_list)):
    df_list[i] = df_list[i].iloc[:, 1:-1]
    df_list[i].GENDER.replace("M", 1, inplace=True)
    df_list[i].GENDER.replace("F", 0, inplace=True)

In [48]:
df_list_1 = []
for i in range(len(df_list)):
    try:
        df_list_1.append(
            df_list[i][["GENDER", "AGE", "ICULOS", "Heart Rate", "RR", "SaO2", "LABEL"]]
        )
    except KeyError:
        continue

In [49]:
len(df_list_1)

0

In [47]:
df_list[0]

,WBC,Creatinine,Glucose (serum),Magnesium,Alkaline Phosphate,Lactic Acid,PTT,ABP-M,RR,SaO2,GENDER
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,91.0,1
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,93.0,1
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,85.0,1
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,93.0,1
4,15.3,0.9,126.0,1.9,NaN,NaN,28.0,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...
124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,93.0,1
125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0,92.0,1
126,NaN,NaN,NaN,NaN,NaN,NaN,56.9,NaN,NaN,NaN,1
127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.0,95.0,1


In [ ]:
# 用所有数据的平均值填充
# cell001_1
s = np.zeros(40).tolist()
c = np.zeros(40).tolist()

for e in df_list:
    s = s + e.sum()[:-1]
    c = c + e.count()[:-1]

a = s.div(c)

# 当不能用线性填充时,填充最近的,再不行用平均值填充
# cell001_2
threshold = 0  # the healthy-sepsis ratio threshold
data = []
i = 0  # counter for extra healthy data samples
nHealthySamples = 0
for e in df_list:
    t = e.iloc[:, -1][e.iloc[:, -1] == 1].sum()
    l = len(e)
    if t / l > threshold:
        e.interpolate(inplace=True, limit_direction="both")
        e.interpolate(inplace=True, method="nearest")
        na_cols = e.columns[e.isna().any()]
        for col in na_cols:
            e.loc[e[col].isnull(), col] = a[col]
        data.append(e)